In [1]:
import numpy  as np
import pandas as pd
from metrics import MAE
from tqdm import tqdm

In [2]:
X_train = pd.read_pickle("../../../Datasets/final/X_train.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
c = X_train.columns.tolist()
X_val = pd.read_pickle("../../../Datasets/final/X_val.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
X_test = pd.read_pickle("../../../Datasets/final/X_test.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
y_train = pd.read_pickle("../../../Datasets/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../../../Datasets/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../../../Datasets/final/y_test.pkl").to_numpy()

In [3]:
X_train.insert(X_train.shape[1], "rating", y_train.T[0])

In [4]:
category_columns = [i for i in c if "category" in i]

In [5]:
mean_per_category_dict = {}
for category in category_columns:
    category_df = X_train.groupby(category).agg({"rating": "mean"}).reset_index()
    mean_per_category_dict[category] = category_df[category_df[category] == 1]["rating"].item()

In [6]:
mean_per_category_dict

{'final_category_AssocOrgs': 3.195639363582793,
 'final_category_Entertainment': 3.7149496514329976,
 'final_category_Legal': 3.92,
 'final_category_Medical': 3.423076923076923,
 'final_category_Others': 3.8192002551528925,
 'final_category_Public': 3.257985257985258,
 'final_category_Restaurant': 3.872157861234882,
 'final_category_School': 3.758250191864927,
 'final_category_ShopsStores': 3.633212649496388,
 'final_category_Venues': 4.029755579171095}

In [7]:
y_pred_val = []
for i in tqdm(range(len(X_val))):
    for j in range(len(X_val.iloc[i][category_columns])):
        if X_val.iloc[i][category_columns][j] == 1:
            y_pred_val.append(mean_per_category_dict[category_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [03:17<00:00, 816.34it/s]


In [8]:
MAE(predictions=y_pred_val, labels=y_val)[0]

1.0720331732067656

In [9]:
y_pred = []
for i in tqdm(range(len(X_test))):
    for j in range(len(X_test.iloc[i][category_columns])):
        if X_test.iloc[i][category_columns][j] == 1:
            y_pred.append(mean_per_category_dict[category_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [03:20<00:00, 806.87it/s]


In [10]:
MAE(predictions=y_pred, labels=y_test)[0]

1.0670154491063117